In [1]:
import pandas as pd
df=pd.read_csv(r'D:\Data/sentiment_cleaned_data.csv',encoding='latin-1')
df=df.drop(columns=['Unnamed: 0','length'])
df.head()

,target,transformed_tweet
0,0,switchfoot http twitpic com zl awww bummer sho...
1,0,upset updat facebook text might cri result sch...
2,0,kenichan dive mani time ball manag save rest g...
3,0,whole bodi feel itchi like fire
4,0,nationwideclass behav mad see


In [2]:
df.shape

(1599505, 2)

In [3]:
df=df.sample(200000)

In [4]:
X=list(df['transformed_tweet'])

In [5]:
y=list(df['target'])

In [6]:
# y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size = 0.30, random_state = 42)

In [8]:
import numpy as np
np.array(y_test).shape

(60000,)

In [9]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

c:\Users\sunne\anaconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
train_encodings = tokenizer(X_train, truncation=True, padding=True,max_length=50)
test_encodings = tokenizer(X_test, truncation=True, padding=True)
val_encodings=tokenizer(X_val, truncation=True, padding=True)

In [11]:
np.array(test_encodings['input_ids']).shape

(60000, 55)

In [12]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))

In [13]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
     eval_steps=100
)

In [14]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
   
)

trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

UnknownError: {{function_node __wrapped__SaveV2_dtypes_318_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to WriteFile: ./results\checkpoint\ckpt-136_temp/part-00000-of-00001.data-00000-of-00001.tempstate16454293570138759653 : There is not enough space on the disk.
; operation in progress [Op:SaveV2]

In [16]:
y_pred=model.predict(test_dataset)

60000/60000 [==============================] - 700s 12ms/step


In [17]:
y_pred

TFSequenceClassifierOutput(loss=None, logits=array([[ 0.7840634 , -0.9796984 ],
       [-0.01579132, -0.1339802 ],
       [-0.02237193, -0.12863237],
       ...,
       [-1.0138199 ,  0.45003352],
       [ 0.64735   , -0.82875735],
       [ 0.03371293, -0.17347728]], dtype=float32), hidden_states=None, attentions=None)

In [18]:
y_pred=y_pred[0]

In [19]:
# Create a mask for the condition (first element > second element)
mask = y_pred[:, 0] < y_pred[:, 1]

# Apply the mask to create the binary predictions
y_pred = np.where(mask, 1,0)

In [20]:
from sklearn import metrics

def model_report(y_test,y_pred):
    print(f'----Confusion matrix---- \n \n{metrics.confusion_matrix(y_test,y_pred)}\n')
    print(f'Acccuracy Score : {metrics.accuracy_score(y_test,y_pred)}\n')
    print(metrics.classification_report(y_test,y_pred))

# y_test=df_test['target']
model_report(y_test,y_pred)

----Confusion matrix---- 
 
[[22637  7291]
 [ 8660 21412]]

Acccuracy Score : 0.73415

              precision    recall  f1-score   support

           0       0.72      0.76      0.74     29928
           1       0.75      0.71      0.73     30072

    accuracy                           0.73     60000
   macro avg       0.73      0.73      0.73     60000
weighted avg       0.73      0.73      0.73     60000



In [21]:
trainer.save_model('senti_model')